In [2]:
pip install selenium

   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
    --------------------------------------- 0.2/9.4 MB 3.9 MB/s eta 0:00:03
   - -------------------------------------- 0.4/9.4 MB 4.2 MB/s eta 0:00:03
   -- ------------------------------------- 0.6/9.4 MB 4.4 MB/s eta 0:00:03
   --- ------------------------------------ 0.8/9.4 MB 4.3 MB/s eta 0:00:03
   ---- ----------------------------------- 1.0/9.4 MB 4.3 MB/s eta 0:00:02
   ----- ---------------------------------- 1.2/9.4 MB 4.5 MB/s eta 0:00:02
   ----- ---------------------------------- 1.2/9.4 MB 4.5 MB/s eta 0:00:02
   ----- ---------------------------------- 1.2/9.4 MB 4.5 MB/s eta 0:00:02
   ----- ---------------------------------- 1.2/9.4 MB 4.5 MB/s eta 0:00:02
   ----- ---------------------------------- 1.2/9.4 MB 4.5 MB/s eta 0:00:02
   ----- ---------------------------------- 1.3/9.4 MB 2.6 MB/s eta 0:00:04
   ----- ---------------------------------- 1.3/9.4 MB 2.4 MB/s eta 0:00:04
   ------ ---------

In [2]:
pip install kiwipiepy

     ---------------------------------------- 0.0/34.7 MB ? eta -:--:--
     --------------------------------------- 0.0/34.7 MB 991.0 kB/s eta 0:00:36
     ---------------------------------------- 0.2/34.7 MB 2.1 MB/s eta 0:00:17
     ---------------------------------------- 0.4/34.7 MB 3.3 MB/s eta 0:00:11
      --------------------------------------- 0.6/34.7 MB 3.3 MB/s eta 0:00:11
      --------------------------------------- 0.8/34.7 MB 3.9 MB/s eta 0:00:09
     - -------------------------------------- 1.0/34.7 MB 3.9 MB/s eta 0:00:09
     - -------------------------------------- 1.2/34.7 MB 4.0 MB/s eta 0:00:09
     - -------------------------------------- 1.5/34.7 MB 4.3 MB/s eta 0:00:08
     - -------------------------------------- 1.6/34.7 MB 4.3 MB/s eta 0:00:08
     -- ------------------------------------- 1.9/34.7 MB 4.4 MB/s eta 0:00:08
     -- ------------------------------------- 2.2/34.7 MB 4.7 MB/s eta 0:00:07
     -- ------------------------------------- 2.4/34.7 MB 

In [ ]:
pip

In [1]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import random
import requests
from kiwipiepy import Kiwi
import time
from quickspacer import Spacer
class AdverbAugmentation():
    def __init__(self):
        self.kiwi = Kiwi()
        self.spacing = Spacer().space
    def _adverb_detector(self, sentence):

        # POS info
        pos_list = [(x[0], x[1]) for x in self.kiwi.tokenize(sentence)] # (token, pos)
        
        adverb_list = []
        for pos in pos_list:
            if pos[1] == "MAG" and len(pos[0]) > 1: # 1음절 부사는 제외함.
                adverb_list.append(pos[0])
        return adverb_list

    def _get_gloss(self, word):
        res = requests.get("https://dic.daum.net/search.do?q=" + word, timeout=5)
        time.sleep(random.uniform(0.5,2.5))
        soup = BeautifulSoup(res.content, "html.parser")
        try:
            # 첫 번째 뜻풀이.
            meaning = soup.find('span', class_='txt_search')
        except AttributeError:
            return word
        if not meaning:
            return word
        
        # parsing 결과에서 한글만 추출
        meaning = re.findall('[가-힣]+', str(meaning))
        meaning = ' '.join(meaning)
        
        # 띄어쓰기 오류 교정 (위 에 -> 위에)
        # meaning = spell_checker.check(meaning).as_dict()['checked'].strip()
        meaning = self.spacing([meaning.replace(" ", "")])
        return meaning[0].strip()
    
    def adverb_gloss_replacement(self, sentence):
        adverb_list = self._adverb_detector(sentence)
        if adverb_list:
            # 부사들 중에서 1개만 랜덤으로 선택합니다.
            adverb = random.choice(adverb_list)
            try:
                gloss = self._get_gloss(adverb)
                sentence = sentence.replace(adverb, gloss)
            except:
                print('except: ', sentence)
                pass
        return sentence

In [2]:
adverb_aug = AdverbAugmentation()
adverb_gloss_replacement = adverb_aug.adverb_gloss_replacement

In [6]:
sentence = "배가 너무 아파요 선생님ㅠㅠ"
adverb_gloss_replacement(sentence)

'배가 정해진 정도나 한계에 지나치게 아파요 선생님ㅠㅠ'

In [ ]:
import pandas as pd
import random
import re
from bs4 import BeautifulSoup
import requests
from kiwipiepy import Kiwi
import time
from quickspacer import Spacer

class AdverbAugmentation:
    def __init__(self):
        self.kiwi = Kiwi()
        self.spacing = Spacer().space

    def _adverb_detector(self, sentence):
        pos_list = [(x[0], x[1]) for x in self.kiwi.tokenize(sentence)] # (token, pos)
        adverb_list = []
        for pos in pos_list:
            if pos[1] == "MAG" and len(pos[0]) > 1: # 1음절 부사는 제외함.
                adverb_list.append(pos[0])
        return adverb_list

    def _get_gloss(self, word):
        res = requests.get("https://dic.daum.net/search.do?q=" + word, timeout=5)
        time.sleep(random.uniform(0.5, 2.5))
        soup = BeautifulSoup(res.content, "html.parser")
        try:
            meaning = soup.find('span', class_='txt_search')
        except AttributeError:
            return word
        if not meaning:
            return word
        meaning = re.findall('[가-힣]+', str(meaning))
        meaning = ' '.join(meaning)
        meaning = self.spacing([meaning.replace(" ", "")])
        return meaning[0].strip()

    def adverb_gloss_replacement(self, sentence):
        adverb_list = self._adverb_detector(sentence)
        if adverb_list:
            adverb = random.choice(adverb_list)
            try:
                gloss = self._get_gloss(adverb)
                sentence = sentence.replace(adverb, gloss)
            except Exception as e:
                print(f'Exception: {e}, sentence: {sentence}')
                pass
        return sentence

# CSV 파일 경로
input_file = 'augmented_output.csv'

# CSV 파일을 읽어 데이터프레임으로 변환
df = pd.read_csv(input_file, encoding='utf-8')

# Adverb Augmentation 객체 생성
augmentor = AdverbAugmentation()

# 새로운 행을 추가할 리스트
new_rows = []

# 데이터프레임의 각 행에 대해 증강을 수행하고 새로운 행을 추가합니다.
for index, row in df.iterrows():
    new_input = augmentor.adverb_gloss_replacement(row['inputs'])
    new_response = augmentor.adverb_gloss_replacement(row['response'])
    new_rows.append({"inputs": new_input, "response": new_response})

# 원본 데이터프레임에 새로운 행을 추가합니다.
augmented_df = pd.DataFrame(new_rows)
combined_df = pd.concat([df, augmented_df], ignore_index=True)

# 증강된 데이터를 새로운 CSV 파일로 저장
output_file = 'Adverb_augmented_output.csv'
combined_df.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"Augmented data saved to {output_file}")
